# PromptTemplate

---

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. PromptTemplate 사용하기

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

### 2-1. 방법 1 : `from_template()` 메서드 사용해 PromptTemplate 객체 생성하기

In [3]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는?"

prompt = PromptTemplate.from_template(template)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는?')

변수에 값을 전달해서 Prompt 완성

In [4]:
prompt.format(country="대한민국")

'대한민국의 수도는?'

PromptTemplate을 활용해 Chain 생성

In [5]:
chain = prompt | llm

In [7]:
chain.invoke({"country" : "대한민국"})

AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 13, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-846687b1-cbad-4dc6-8068-fa33725cf4ea-0', usage_metadata={'input_tokens': 13, 'output_tokens': 9, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 2-2. 방법 2 : PromptTemplate 객체 생성과 동시에 Prompt 생성하기

- 추가 유효성 검사 과정을 거치기 위해 `input_variables`를 명시적으로 지정해주는 것이 좋다
    - `input_variables`에 지정한 변수는 해당하는 key값에 값이 들어오지 않으면 오류를 발생해줌

In [9]:
template = "{country}의 수도는?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는?')

`partial_variables` 파라미터를 활용해서 사용자가 값을 전달하지 않아도 template의 특정 변수값에 자동으로 값을 채우도록 할 수 있음
- 주로 실행 중간 단계에서 template의 특정 값이 결정되어 따로 입력으로 넣어줄 필요가 없는 경우 사용한다.

In [14]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template = template,
    input_variables=["country1"],
    partial_variables={
        "country2" : "미국"
    }
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [17]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

이와 비슷하게 `.partial()` 메서드를 사용하면 변수가 2개 이상인 template에서 특정 변수에 값만 미리 채워넣을 수 있음

In [18]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate.from_template(template)

prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [19]:
prompt.partial(country2="대한민국")

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '대한민국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

### 2-3. `partial_variables` : 부분 변수 채움

- partial 의 일반적인 용도는 template의 특정 변수를 함수 실행으로 채우고 싶을 때 사용함
    - 항상 공통된 방식으로 가져오고 싶은 변수 가 있는 경우 많이 사용함.
    - 대표적인 예가 날짜나 시간
- 항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해보면,
    - 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거로움
    - 이 경우 항상 현재 날짜를 반환하는 함수 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리함

In [26]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%B %d")

prompt = PromptTemplate(
    template = "오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일도 함께 나열해주세요.",
    input_variables=["n"],
    partial_variables={
        "today" : get_today
    }
)

In [27]:
prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x0000020B5F85F4C0>}, template='오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일도 함께 나열해주세요.')

In [28]:
prompt.format(n=3)

'오늘 날짜는 February 27 입니다. 오늘이 생일인 유명인 3명을 나열해주세요. 생년월일도 함께 나열해주세요.'

In [31]:
chain = prompt | llm

print(chain.invoke({"n" : 3}).content)

February 27일에 태어난 유명인 중 세 명을 소개하겠습니다.

1. 엘리자베스 테일러 (Elizabeth Taylor) - 1932년 2월 27일
   - 영국 출신의 할리우드 여배우로, 그녀의 아름다움과 연기력으로 많은 사랑을 받았습니다. 대표작으로는 "클레오파트라" 등이 있습니다.

2. 조쉬 그로반 (Josh Groban) - 1981년 2월 27일
   - 미국의 가수이자 배우로, 그의 풍부한 성량과 감성적인 발라드로 유명합니다. "You Raise Me Up" 등의 히트곡이 있습니다.

3. 케이트 마라 (Kate Mara) - 1983년 2월 27일
   - 미국의 배우로, "하우스 오브 카드"와 "판타스틱 4" 등의 작품에 출연했습니다. 그녀는 마라 패밀리의 일원으로도 잘 알려져 있습니다.

이 외에도 많은 유명인들이 이 날 태어났습니다.


In [32]:
print(chain.invoke({"today" : "Jan 02", "n" : 3}).content)

1월 2일에 태어난 유명인 중 일부는 다음과 같습니다:

1. **아이작 아시모프 (Isaac Asimov)** - 1920년 1월 2일 출생. 러시아 태생의 미국 작가이자 생화학자이며, 과학 소설과 과학 서적을 다수 집필한 것으로 유명합니다.

2. **큐반 굿잉 주니어 (Cuba Gooding Jr.)** - 1968년 1월 2일 출생. 미국의 배우로, 영화 "제리 맥과이어"에서의 연기로 아카데미 남우조연상을 수상했습니다.

3. **케이트 보스워스 (Kate Bosworth)** - 1983년 1월 2일 출생. 미국의 배우로, 영화 "블루 크러쉬"와 "슈퍼맨 리턴즈" 등에 출연했습니다.

이 외에도 여러 유명인들이 1월 2일에 태어났습니다.
